# **2494. Merge Overlapping Events in the Same Hall**

**Table: HallEvents**
``` sql
+-------------+------+
| Column Name | Type |
+-------------+------+
| hall_id     | int  |
| start_day   | date |
| end_day     | date |
+-------------+------+
```
This table may contain duplicates rows.

Each row of this table indicates the start day and end day of an event and the hall in which the event is held.
 

Write a solution to merge all the overlapping events that are held in the same hall. Two events overlap if they have at least one day in common.

Return the result table in any order.

The result format is in the following example.

 

**Example 1:**

**Input:** 
**HallEvents table:**
``` sql
+---------+------------+------------+
| hall_id | start_day  | end_day    |
+---------+------------+------------+
| 1       | 2023-01-13 | 2023-01-14 |
| 1       | 2023-01-14 | 2023-01-17 |
| 1       | 2023-01-18 | 2023-01-25 |
| 2       | 2022-12-09 | 2022-12-23 |
| 2       | 2022-12-13 | 2022-12-17 |
| 3       | 2022-12-01 | 2023-01-30 |
+---------+------------+------------+
```
**Output:**
``` sql
+---------+------------+------------+
| hall_id | start_day  | end_day    |
+---------+------------+------------+
| 1       | 2023-01-13 | 2023-01-17 |
| 1       | 2023-01-18 | 2023-01-25 |
| 2       | 2022-12-09 | 2022-12-23 |
| 3       | 2022-12-01 | 2023-01-30 |
+---------+------------+------------+
```
**Explanation:** There are three halls.

**Hall 1:**

- The two events ["2023-01-13", "2023-01-14"] and ["2023-01-14", "2023-01-17"] overlap. We merge them in one event ["2023-01-13", "2023-01-17"].

- The event ["2023-01-18", "2023-01-25"] does not overlap with any other event, so we leave it as it is.

**Hall 2:**

- The two events ["2022-12-09", "2022-12-23"] and ["2022-12-13", "2022-12-17"] overlap. We merge them in one event ["2022-12-09", "2022-12-23"].

**Hall 3:**

- The hall has only one event, so we return it. Note that we only consider the events of each hall separately.

---
**Solution-1:**
``` sql
WITH rollingcalcs AS (
    SELECT 
    hall_id, 
    start_day, 
    end_day, 
    MIN(start_day) OVER(PARTITION BY hall_id ORDER BY start_day, end_day ROWS BETWEEN UNBOUNDED PRECEDING and 1 PRECEDING) as min_start_day,
    MAX(end_day) OVER(PARTITION BY hall_id ORDER BY start_day, end_day ROWS BETWEEN UNBOUNDED PRECEDING and 1 PRECEDING) as max_end_day,
    (CASE WHEN start_day > MAX(end_day) OVER(PARTITION BY hall_id ORDER BY start_day, end_day ROWS BETWEEN UNBOUNDED PRECEDING and 1 PRECEDING)
        or MAX(end_day) OVER(PARTITION BY hall_id ORDER BY start_day, end_day ROWS BETWEEN UNBOUNDED PRECEDING and 1 PRECEDING) IS NULL
        THEN 1 
        ELSE 0
        END) as overlaps
    FROM HallEvents
), 
addgroups as (
    SELECT 
    hall_id,
    start_day, 
    end_day, 
    SUM(overlaps) OVER(PARTITION BY hall_id ORDER BY start_day, end_day) as summed
    FROM rollingcalcs 
)

SELECT hall_id, MIN(start_day) as start_day, MAX(end_day) as end_day
FROM addgroups
GROUP BY hall_id, summed
ORDER BY hall_id, start_day
```

---
**Solution-1:**
``` sql
with cte as (
    select distinct hall_id, start_day, end_day,
    row_number() over (order by hall_id, start_day, end_day) as row_num
    from hallevents
),
cte1 as (
    select hall_id, start_day, end_day, row_num,
    lag(end_day, 1) over (partition by hall_id order by row_num) as prev_end
    from cte
),
cte2 as (
    select hall_id, start_day, end_day, row_num,
    prev_end,
    max(prev_end) over (partition by hall_id order by row_num) as max_prev_end
    from cte1
),
cte3 as (
    select hall_id, start_day, end_day, row_num,
    prev_end,
    max_prev_end,
    (case when start_day <= max_prev_end then 0 else 1 end) as is_new
    from cte2
),
cte4 as (
    select hall_id, start_day, end_day, row_num,
    prev_end,
    max_prev_end,
    is_new,
    sum(is_new) over (partition by hall_id order by row_num) as group_num
    from cte3
)

select hall_id, min(start_day) as start_day, max(end_day) as end_day
from cte4
group by hall_id, group_num
```
---